In [2]:
import tensorflow as tf
import numpy as np

In [3]:
bandits = [0.2,0,-0.2,-5]
num_bandits = len(bandits)
    #define four bandits - if rand number is greater than ^ val, get positive reward

def pullBandit(bandit):
    result = np.random.randn(1)
    
    if result > bandit:
        return 1
    else:
        return -1

In [20]:
tf.reset_default_graph()
    #reset the network graph

weights = tf.Variable(tf.ones([num_bandits]))
    #1x4 array of weights - one for each bandit
chosen_action = tf.argmax(weights,0)
    #returns largest weight along axis 0
    
reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
    #placeholder for the reward from each pull
action_holder = tf.placeholder(shape=[1],dtype=tf.int32)
    #placeholder for action
responsible_weight = tf.slice(weights, action_holder, [1])
    #takes the [action_holder]'th slice from weights - action_holder selects which weight 
    
loss = -(tf.log(responsible_weight)*reward_holder)
    #the loss function is the weight for the action we took * the reward obtained
    #bad rewards are negative -> loss positive
    #good rewards are positive -> loss negative
    #minimize loss -> high weight on good pulls makes loss v negative
    #q. is loss bounded? why not make the weight 28937493028
    
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
update = optimizer.minimize(loss)


In [22]:
total_episodes = 1000
    #train for 1000 episodes
total_reward = np.zeros(num_bandits)
    #scoreboard init
e = 0.1
    #explore /exploit
    
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        if np.random.rand(1) < e:
            action = np.random.randint(num_bandits)
        else: 
            action = sess.run(chosen_action)
                #run tf session to get chosen action
        
        reward = pullBandit(bandits[action])
            #execute action
        
        
        _, resp, ww = sess.run([update, responsible_weight, weights], feed_dict={reward_holder:[reward], action_holder:[action]})
            #train:
            #minimize loss w.r.t weight
            #if we change the weight, it will change the action selected which changes the reward attained
            #responsible weight is the weight that led to this action and reward
            #weight changes how responsible weight is chosen
            #q. what if the update does not change the weights enough to select another action?
            
        #Update our running tally of scores.
        total_reward[action] += reward
        print (str(action) + " " + str(total_reward) + " " + str(ww))
        i+=1
print ("The agent thinks bandit " + str(np.argmax(ww)+1) + " is the most promising....")
if np.argmax(ww) == np.argmax(-np.array(bandits)):
    print ("...and it was right!")
else:
    print ("...and it was wrong!")

0 [-1.  0.  0.  0.] [ 0.99900001  1.          1.          1.        ]
1 [-1. -1.  0.  0.] [ 0.99900001  0.99900001  1.          1.        ]
2 [-1. -1.  1.  0.] [ 0.99900001  0.99900001  1.00100005  1.        ]
2 [-1. -1.  2.  0.] [ 0.99900001  0.99900001  1.00199902  1.        ]
2 [-1. -1.  1.  0.] [ 0.99900001  0.99900001  1.001001    1.        ]
2 [-1. -1.  2.  0.] [ 0.99900001  0.99900001  1.00199997  1.        ]
2 [-1. -1.  3.  0.] [ 0.99900001  0.99900001  1.00299799  1.        ]
2 [-1. -1.  2.  0.] [ 0.99900001  0.99900001  1.00200093  1.        ]
2 [-1. -1.  1.  0.] [ 0.99900001  0.99900001  1.00100291  1.        ]
2 [-1. -1.  2.  0.] [ 0.99900001  0.99900001  1.00200188  1.        ]
3 [-1. -1.  2.  1.] [ 0.99900001  0.99900001  1.00200188  1.00100005]
0 [ 0. -1.  2.  1.] [ 1.00000107  0.99900001  1.00200188  1.00100005]
2 [ 0. -1.  3.  1.] [ 1.00000107  0.99900001  1.0029999   1.00100005]
2 [ 0. -1.  2.  1.] [ 1.00000107  0.99900001  1.00200284  1.00100005]
2 [ 0. -1.  1.  1.] 

1 [   1.   -3.    0.  125.] [ 1.00100303  0.99699903  1.00000775  1.11808407]
3 [   1.   -3.    0.  126.] [ 1.00100303  0.99699903  1.00000775  1.1189785 ]
3 [   1.   -3.    0.  127.] [ 1.00100303  0.99699903  1.00000775  1.11987221]
3 [   1.   -3.    0.  128.] [ 1.00100303  0.99699903  1.00000775  1.12076521]
3 [   1.   -3.    0.  129.] [ 1.00100303  0.99699903  1.00000775  1.12165749]
3 [   1.   -3.    0.  130.] [ 1.00100303  0.99699903  1.00000775  1.12254906]
3 [   1.   -3.    0.  131.] [ 1.00100303  0.99699903  1.00000775  1.12343991]
3 [   1.   -3.    0.  132.] [ 1.00100303  0.99699903  1.00000775  1.12433004]
3 [   1.   -3.    0.  133.] [ 1.00100303  0.99699903  1.00000775  1.12521946]
3 [   1.   -3.    0.  134.] [ 1.00100303  0.99699903  1.00000775  1.12610817]
2 [   1.   -3.    1.  134.] [ 1.00100303  0.99699903  1.0010078   1.12610817]
3 [   1.   -3.    1.  135.] [ 1.00100303  0.99699903  1.0010078   1.12699616]
3 [   1.   -3.    1.  136.] [ 1.00100303  0.99699903  1.0010078 

3 [   1.   -1.    1.  259.] [ 1.00100505  0.99900508  1.0010097   1.23215616]
0 [   0.   -1.    1.  259.] [ 1.00000608  0.99900508  1.0010097   1.23215616]
3 [   0.   -1.    1.  260.] [ 1.00000608  0.99900508  1.0010097   1.23296773]
3 [   0.   -1.    1.  261.] [ 1.00000608  0.99900508  1.0010097   1.23377883]
3 [   0.   -1.    1.  262.] [ 1.00000608  0.99900508  1.0010097   1.23458934]
3 [   0.   -1.    1.  263.] [ 1.00000608  0.99900508  1.0010097   1.23539937]
3 [   0.   -1.    1.  264.] [ 1.00000608  0.99900508  1.0010097   1.2362088 ]
3 [   0.   -1.    1.  265.] [ 1.00000608  0.99900508  1.0010097   1.23701775]
3 [   0.   -1.    1.  266.] [ 1.00000608  0.99900508  1.0010097   1.23782611]
0 [  -1.   -1.    1.  266.] [ 0.99900609  0.99900508  1.0010097   1.23782611]
3 [  -1.   -1.    1.  267.] [ 0.99900609  0.99900508  1.0010097   1.23863399]
3 [  -1.   -1.    1.  268.] [ 0.99900609  0.99900508  1.0010097   1.23944128]
3 [  -1.   -1.    1.  269.] [ 0.99900609  0.99900508  1.0010097 

3 [   0.    2.    3.  398.] [ 1.00000906  1.00200605  1.00300765  1.34025788]
3 [   0.    2.    3.  399.] [ 1.00000906  1.00200605  1.00300765  1.34100401]
3 [   0.    2.    3.  400.] [ 1.00000906  1.00200605  1.00300765  1.34174967]
3 [   0.    2.    3.  401.] [ 1.00000906  1.00200605  1.00300765  1.34249496]
3 [   0.    2.    3.  402.] [ 1.00000906  1.00200605  1.00300765  1.3432399 ]
3 [   0.    2.    3.  403.] [ 1.00000906  1.00200605  1.00300765  1.34398437]
3 [   0.    2.    3.  404.] [ 1.00000906  1.00200605  1.00300765  1.34472847]
3 [   0.    2.    3.  405.] [ 1.00000906  1.00200605  1.00300765  1.3454721 ]
3 [   0.    2.    3.  406.] [ 1.00000906  1.00200605  1.00300765  1.34621537]
3 [   0.    2.    3.  407.] [ 1.00000906  1.00200605  1.00300765  1.34695816]
3 [   0.    2.    3.  408.] [ 1.00000906  1.00200605  1.00300765  1.3477006 ]
3 [   0.    2.    3.  409.] [ 1.00000906  1.00200605  1.00300765  1.34844255]
3 [   0.    2.    3.  410.] [ 1.00000906  1.00200605  1.00300765

3 [  -5.    2.    3.  539.] [ 0.995       1.00200713  1.00300872  1.44165301]
3 [  -5.    2.    3.  540.] [ 0.995       1.00200713  1.00300872  1.44234669]
3 [  -5.    2.    3.  541.] [ 0.995       1.00200713  1.00300872  1.44304001]
3 [  -5.    2.    3.  542.] [ 0.995       1.00200713  1.00300872  1.44373298]
3 [  -5.    2.    3.  543.] [ 0.995       1.00200713  1.00300872  1.44442558]
3 [  -5.    2.    3.  544.] [ 0.995       1.00200713  1.00300872  1.44511795]
3 [  -5.    2.    3.  545.] [ 0.995       1.00200713  1.00300872  1.44580996]
0 [  -4.    2.    3.  545.] [ 0.99600506  1.00200713  1.00300872  1.44580996]
3 [  -4.    2.    3.  546.] [ 0.99600506  1.00200713  1.00300872  1.44650161]
3 [  -4.    2.    3.  547.] [ 0.99600506  1.00200713  1.00300872  1.44719291]
3 [  -4.    2.    3.  548.] [ 0.99600506  1.00200713  1.00300872  1.44788384]
3 [  -4.    2.    3.  549.] [ 0.99600506  1.00200713  1.00300872  1.44857454]
1 [  -4.    3.    3.  549.] [ 0.99600506  1.00300515  1.00300872

3 [  -6.    5.    6.  694.] [ 0.99399704  1.00499904  1.0059967   1.54545534]
3 [  -6.    5.    6.  695.] [ 0.99399704  1.00499904  1.0059967   1.5461024 ]
3 [  -6.    5.    6.  696.] [ 0.99399704  1.00499904  1.0059967   1.54674923]
3 [  -6.    5.    6.  697.] [ 0.99399704  1.00499904  1.0059967   1.54739571]
3 [  -6.    5.    6.  698.] [ 0.99399704  1.00499904  1.0059967   1.54804194]
1 [  -6.    4.    6.  698.] [ 0.99399704  1.004004    1.0059967   1.54804194]
3 [  -6.    4.    6.  699.] [ 0.99399704  1.004004    1.0059967   1.54868793]
0 [  -5.    4.    6.  699.] [ 0.9950031   1.004004    1.0059967   1.54868793]
3 [  -5.    4.    6.  700.] [ 0.9950031   1.004004    1.0059967   1.54933369]
3 [  -5.    4.    6.  701.] [ 0.9950031   1.004004    1.0059967   1.54997909]
3 [  -5.    4.    6.  702.] [ 0.9950031   1.004004    1.0059967   1.55062425]
3 [  -5.    4.    6.  703.] [ 0.9950031   1.004004    1.0059967   1.55126917]
3 [  -5.    4.    6.  704.] [ 0.9950031   1.004004    1.0059967 

3 [  -5.    3.    7.  828.] [ 0.99500519  1.00301003  1.00699174  1.62987268]
3 [  -5.    3.    7.  829.] [ 0.99500519  1.00301003  1.00699174  1.63048625]
3 [  -5.    3.    7.  830.] [ 0.99500519  1.00301003  1.00699174  1.63109958]
3 [  -5.    3.    7.  831.] [ 0.99500519  1.00301003  1.00699174  1.63171268]
3 [  -5.    3.    7.  832.] [ 0.99500519  1.00301003  1.00699174  1.63232553]
3 [  -5.    3.    7.  833.] [ 0.99500519  1.00301003  1.00699174  1.63293815]
3 [  -5.    3.    7.  834.] [ 0.99500519  1.00301003  1.00699174  1.63355052]
3 [  -5.    3.    7.  835.] [ 0.99500519  1.00301003  1.00699174  1.63416266]
3 [  -5.    3.    7.  836.] [ 0.99500519  1.00301003  1.00699174  1.63477457]
3 [  -5.    3.    7.  837.] [ 0.99500519  1.00301003  1.00699174  1.63538623]
3 [  -5.    3.    7.  838.] [ 0.99500519  1.00301003  1.00699174  1.63599765]
3 [  -5.    3.    7.  839.] [ 0.99500519  1.00301003  1.00699174  1.63660896]
3 [  -5.    3.    7.  840.] [ 0.99500519  1.00301003  1.00699174